### This is basically Fast.ai Lesson 5 with extra comments

In [36]:
# Load in theano 
from theano.sandbox import cuda
from imp import reload
%matplotlib inline

# Loads the utils function
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [37]:
# Sets up the pathing
model_path = '/home/jd/aidata/imdb/'

## Set up sound alert
from IPython.display import Audio, display
from imp import reload
import time

def allDone():
 #   self.showtraceback((etype, value, tb), tb_offset=tb_offset)
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

## Pull in the required data

Keras has some datasets included. https://keras.io/datasets/

In [38]:
# Keras allows simple inclusion of imdb from the Keras datasets. 
from keras.datasets import imdb
idx = imdb.get_word_index()

Word List

In [39]:
# Sorts the idx according to 
idx_arr = sorted(idx, key=idx.get)
# Prints top words from the index through 10
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [40]:
# Combines the index number with the word i.e.
# 1: 'the',
# 2: 'and',
# 3: 'a',
# 4: 'of',
# 5: 'to',
idx2word = {v: k for k, v in idx.items()}

### Pickle
Read a string from the open file object file and interpret it as a pickle data stream. https://docs.python.org/2/library/pickle.html

In [41]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
# X-train - Array of Reviews in the training data
# labels_train - what the bias is
# x_test - Array of Reviews in the test data
# labels_test - what the bias is for test (0,1)
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [42]:
# Length of the X_Train
len(x_train)

25000

In [43]:
# This is the very first review in the training data. IDs represents words 
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [44]:
# So we walk through the x_train data to see whats in it
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [45]:
# makes either the trn or test sets to max of 5000
vocab_size = 5000

# new Trn and Test set
trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

### Standardize input

In [46]:
# Look for the length of the comment
lens = np.array(map(len, trn))
type (lens)
# we are supposed to figure out what the minumum, maximum, and mean lengths of the array.
#(lens.max(), lens.min(), lens.mean())

numpy.ndarray

In [47]:
#(lens.max(), lens.min(), lens.mean())

In [48]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [49]:
trn.shape

(25000, 500)

## Create simple models

### Single Layer NN

In [50]:
# Create the Model
model = Sequential([
    # Embedding # is sometimes 32 sometimes more sometimes left
    Embedding(vocab_size, 64, input_length=seq_len),
    # 
    Flatten(),
    # Dense layer
    Dense(100, activation='relu'),
    # Helps prevent overfitting
    Dropout(0.5),
    
    ## Round two!
    Dense(100, activation='relu'),
    # Helps prevent overfitting
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')])

In [51]:
#Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
#model.summary()

In [52]:
### Run the model
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 43s - loss: 0.5010 - acc: 0.7200 - val_loss: 0.3020 - val_acc: 0.8704
Epoch 2/2
25000/25000 [==============================] - 43s - loss: 0.1930 - acc: 0.9277 - val_loss: 0.3217 - val_acc: 0.8693


### Single conv layer with max pooling

In [53]:
## Creating a convultional Model
conv1 = Sequential([
    # Setting up an embedding
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    # putting in dropout to reduce overfit
    Dropout(0.2),
    # Convolutional layer covering 5 words at a time
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    
    MaxPooling1D(),
    # Make it into one large model
    Flatten(),
    # Dense layer 
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [54]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [55]:
#conv1.summary()

In [56]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=128)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 32s - loss: 0.5683 - acc: 0.6668 - val_loss: 0.3108 - val_acc: 0.8743


In [57]:
# Save the weights
#conv1.save_weights(model_path + 'conv1.h5')

In [58]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

In [59]:
# Loads the pickle words and idx
# Encoded into the latin language
def load_glove(loc):
    return (load_array(loc+'.dat'),
            #loads the words
        pickle.load(open(loc+'_words.pkl','rb'),encoding='latin1'),
            #loads the index
        pickle.load(open(loc+'_idx.pkl','rb'),encoding='latin1'))

In [60]:
vecs, words, wordidx = load_glove(model_path+'6B.50d')

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [61]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))
    #Walks through the wordlist to see if it is in the glove
    for i in range(1,len(emb)):
        #
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [62]:
# emb are the weights used
emb = create_emb()

Lets make a model!

In [63]:
# Model
model = Sequential([
    # Embedding as before; But it has weights preset and trainable;
    # This adds in teh embed, attached to weights
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    #Same dropout
    Dropout(0.25),
    #Same 
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [64]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [65]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 60s - loss: 0.1706 - acc: 0.9336 - val_loss: 0.3078 - val_acc: 0.8744
Epoch 2/4
25000/25000 [==============================] - 61s - loss: 0.1607 - acc: 0.9382 - val_loss: 0.3671 - val_acc: 0.8612
Epoch 3/4
25000/25000 [==============================] - 61s - loss: 0.1459 - acc: 0.9431 - val_loss: 0.4715 - val_acc: 0.8292
Epoch 4/4
25000/25000 [==============================] - 60s - loss: 0.1504 - acc: 0.9422 - val_loss: 0.3092 - val_acc: 0.8768


In [66]:
conv1.save_weights(model_path + 'conv1.h5')

In [67]:
conv1.load_weights(model_path + 'conv1.h5')

## Multi Size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' excellent blog post.

In [70]:
# Puts more than one input together
from keras.layers import Merge

In [88]:
# Remember vocab size with 5000
graph_in = Input ((vocab_size, 50))
# list of convolution layers
convs = [ ] 

# runs through fsz 3 times taking convolution1Ds and adding them together
#fsz is the size of the filters?
for fsz in range (3, 7): 
    x = Convolution1D(64, fsz, border_mode='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)

# output is  merged
out = Merge(mode="concat")(convs) 
# Graph is created with both the input (Vocab junk) and the output (merged 7 examples)
graph = Model(graph_in, out) 

[Reshape{2}.0, Reshape{2}.0, Reshape{2}.0, Reshape{2}.0]


In [72]:
# Same as before creates the embedded matrix
emb = create_emb()

In [90]:
# model is set up
# Everything is done in a squential order
model = Sequential ([
    # Embedding is set up, notice weights are pregenerated
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    # graph is called, this is our two input model
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [93]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

NameError: name 'Eve' is not defined

In [92]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 136s - loss: 0.5040 - acc: 0.7331 - val_loss: 0.3223 - val_acc: 0.8678
Epoch 2/2
25000/25000 [==============================] - 136s - loss: 0.3115 - acc: 0.8722 - val_loss: 0.2610 - val_acc: 0.8980


In [76]:
model.layers[0].trainable=False

In [77]:
model.optimizer.lr=1e-5

In [78]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 114s - loss: 0.2765 - acc: 0.8886 - val_loss: 0.2588 - val_acc: 0.8960
Epoch 2/2
25000/25000 [==============================] - 113s - loss: 0.2559 - acc: 0.8960 - val_loss: 0.2562 - val_acc: 0.8967
